In [1]:
import random
from __future__ import division
import svmprediction
reload(svmprediction)
import features
reload(features)
from features import extract_features
from features import encode_categorical_features
from sklearn import svm

(features, labels, _, _) = extract_features(2009, 2014)


177245


In [2]:
(svm_features, encoder) = encode_categorical_features(features)

#classifier = svm.SVC()
#cm = svmprediction.plot_confusion_matrix(classifier,svm_features,labels)

In [ ]:
classifier = svm.SVC()
cm = svmprediction.plot_confusion_matrix(classifier,svm_features,labels)

In [5]:
import dimreduction
reload(dimreduction)
pca = dimreduction.pca(svm_features.toarray(), 3)

[  9.97857167e-01   2.07908811e-03   4.91223361e-05]


In [6]:
reduced_features = pca.transform(svm_features.toarray())

In [ ]:
classifier = svm.SVC()
cm = svmprediction.plot_confusion_matrix(classifier,reduced_features,labels)

In [3]:
#Example

#Train classifier
classifier.fit(svm_features, labels)  

#Predict result for play
X = defaultdict(float)
X['team'] = "GB"
X['opponent'] = "NYG"
X['time'] = 425
X['half'] = 1 
X['position'] = 10
X['half'] = 2
X['togo'] = 10
X['shotgun'] = 0
X['pass'] = 1
X['passlen'] = 'short'
X['side'] = 'left'
X['qbrun'] = 0
X = encoder.transform(X)

y_pred = clf.predict(X)
print prediction

NameError: name 'classifier' is not defined